# Decoradores

Em Python, decoradores são elementos que podem ser usados para alterar o comportamento de funções (decoradores de funções) ou classes (decoradores de classes) sobre os quais são aplicados.

Se definimos um decorador denominado `decor` (por exemplo), para aplicá-lo basta usar a sintaxe `@decor` imediatamente antes da função ou classe que será decorada.

## 1. Alguns decoradores pré-definidos

A linguagem já define diversos decoradores. Alguns importantes são ligados a propriedades e métodos estáticos e de classe.

Por exemplo, podemos definir os métodos estáticos e de classe do exemplo dado anteriormente de forma mais simples como na classe seguinte:

In [ ]:
class A:
    x = 0
    
    def __init__(self, y):
        self.y = y
    
    def normal_method(self):
        return self, self.x, self.y
    
    @staticmethod
    def static_method():
        return A, A.x

    @classmethod
    def class_method(cls, z):
        return cls, cls.x + z

In [ ]:
a = A(2)

In [ ]:
a.normal_method()

In [ ]:
a.static_method()

In [ ]:
a.class_method(3)

Existe também o decorador `property`, usado para definir uma *property* de forma mais simples: Basta colocar esse decorador antes da função que será usada **para acessos de leitura**. O nome da função será o nome do atributo criado. Depois, se você quiser controlar também a escrita, basta usar o decorador `nome_do_atributo.setter` antes da função que será usada **para a escrita** (e que deve ter o mesmo nome do atributo). Veja abaixo redefinição da classe `Positive`.

In [ ]:
class Positive:
    def __init__(self, ini):
        self.value = ini
    
    @property
    def value(self):
        return self._val
    
    @value.setter
    def value(self, x):
        if x <= 0:
            raise ValueError('Value must be positive')
        self._val = x

In [ ]:
p4 = Positive(1)

In [ ]:
p4.value

In [ ]:
p4.value = 5

In [ ]:
p4.value

In [ ]:
p4.value += 2

In [ ]:
p4.value -= 10

In [ ]:
p5 = Positive(-1)

## 2. Definindo seus decoradores

Você também pode definir seus próprios decoradores. 

### 2.1. Decoradores de funções

Para definir um decorador de funções você deve criar um objeto que define o método `__call__`, isto é, ele pode ser chamado como uma função. Esse método irá fazer o processamento necessário no lugar da função que foi decorada.

Para entender o processo, você deve pensar que o código

    @decor
    def f(a, b):
        ...

será traduzido para:

    def f(a, b):
        ...
    f = decor(f)

Portanto o seu decorador deve aceitar como parâmetro a função a ser decorada e retornar uma nova função que pode ser utilizada no lugar dela.

Abaixo definimos a classe `tracer` que pode ser usada como um decorador de funções e, para cada chamada da função, imprime o número de vezes que ela já foi chamada, antes de efetivamente realizar a chamada.

In [ ]:
class tracer:
    def __init__(self, fun):
        self.calls = 0
        self._fun = fun
    def __call__(self, *args, **kargs):
        self.calls += 1
        print(f'(Call number {self.calls} of function {self._fun.__name__})')
        return self._fun(*args, **kargs)

Veja como no `__init__` recebemos uma função (`fun`) que é guardada no objeto (`self._fun`). Depois, quando o objeto criado for chamado (`__call__`) pegamos os argumentos passados e os passamos para `self._fun` (no `return`), mas antes disso incrementamos e mostramos o valor do número de chamadas (`self.calls`).

O uso do decorador é simples:

In [ ]:
@tracer
def f(x, y):
    print('Inside f')
    return x + 2 * y

In [ ]:
f(2, 4)

In [ ]:
f(5, 6)

In [ ]:
f(0, -1); f(5, 2); f(10, 15)

### 2.2. Decorando métodos

O decorador definido acima tem um defeito: ele não serve para decorar métodos de classes. Veja o que acontece no código abaixo:

In [ ]:
class B:
    @tracer
    def f(self):
        return self

In [ ]:
b = B()

In [ ]:
b.f()

O problema é que o atributo `self` incluido nas chamadas de métodos não está sendo corretamente passado. Não adianta passar o `self` recebido pelo `__call__`, **pois este será o objeto do tipo `tracer`, ao invés de ser o objeto desejado do tipo `B`**.

Uma solução simples é redefinir o decorador para usar uma *closure*, como no código abaixo.

In [ ]:
def mtracer(fun):
    def oncall(*args, **kargs):
        oncall.calls += 1
        print(f'Call number {oncall.calls} of function {fun.__name__}')
        return fun(*args, **kargs)
    oncall.calls = 0
    return oncall

Note como aqui o decorador é uma função (e não uma classe), que recebe a função original como parâmetro `fun`. Ele então cria uma nova função, denominada `oncall`, definida internamente (uma *closure*) que faz acesso a `fun` na hora que for chamada. Essa é a função que será chamada, executando o incremento do seu atributo `calls` sempre que for chamada. (Como funções são também objetos, elas podem ter atributos.)

In [ ]:
@mtracer
def g(x, y):
    print('Inside f')
    return x ** y - 1

O código acima será traduzido em

    def g(x, y):
        print('Inside f')
        return x ** y - 1
    g = mtracer(g)

E portanto `g` será substituida pela função `oncall` que faz acesso a `g` (guardado em `oncall.fun`) em seu `return`.

In [ ]:
g(2, 4)

In [ ]:
g(3, 5)

Esse decorador funciona também para métodos, pois não há um objeto adicional no meio do caminho (o método da classe `B` abaixo é substituido pelo decorador).

In [ ]:
class B:
    @mtracer
    def f(self):
        return self

In [ ]:
b = B()

In [ ]:
b.f()

In [ ]:
b.f()

In [ ]:
B.f(b)

### 2.3. Decorando classes

Agora vejamos um exemplo de um decorador de classes.

Para entender decoradores de classes, você precisa se lembrar que um código como:

    @decor
    class A:
        ...

será substituido por:

    class A:
        ...
    A = decor(A)

Portanto, o seu decorador deve retornar uma *classe*, cujos objetos servirão como substitutos para os objetos da classe `A`.

No exemplo abaixo, definimos um decorador que altera a forma de converter objetos da classe decorada para cadeias de caracteres (acrescentando uma mensagem).

In [ ]:
def existentialist(cls):
    class Ex(cls):
        def __init__(self, *args, **kargs):
            cls.__init__(self, *args, **kargs)
        def __repr__(self):
            return cls.__repr__(self) + ' [But life is pointless!]'
    return Ex

Veja como o decorador é uma função que define internamente uma classe e a retorna. Essa classe interna faz referência à classe original (que é a classe base `cls`) e a usa (por meio de herança) para sua implementação.

In [ ]:
@existentialist
class S1:
    def __init__(self, x):
        self.val = x
    def __repr__(self):
        return 'S1(' + str(self.val) + ')'

In [ ]:
s1 = S1(10)

In [ ]:
s1

In [ ]:
@existentialist
class MyList(list):
    pass

In [ ]:
x = MyList()
x.append(1); x.append(3); x.append(7)

In [ ]:
x

### 2.4 Decoradores com parâmetros

Em algumas situações, queremos adaptar o funcionamento de um decorador através de um parâmetro.

Para fazer isso, precisamos definir uma função que retorna um decorador, e então usamos essa função como o decorador.

É fácil de ver que isso é necessário lembrando que
```python
@decor(par)
def f():
    ...
```
será traduzido para
```python
def f():
    ...
f = (decor(par))(f)
```
isto é, `decor(par)` deve retornar uma função (ou objeto funcional) que receberá a função a ser decorada e deve retornar a nova função.

Vejamos um exemplo.

In [ ]:
def tracer(*, after=False):
    def this_tracer(fun):
        def oncall(*args, **kargs):
            oncall.calls += 1
            print(f'Call number {oncall.calls} of function {fun.__name__}')
            val = fun(*args, **kargs)
            if after:
                print(f'Call number {oncall.calls} of function {fun.__name__} returned {val}')
            return val
        oncall.calls = 0
        return oncall
    return this_tracer

In [ ]:
@tracer()
def f(x):
    return 2 * x

f(1)
f(2)
f(3)

In [ ]:
@tracer(after=True)
def g(x):
    return 3 * x

g(1)
g(2)
g(3)

## 3. Um exemplo prático

Para terminar, vamos ver um exemplo de decorador pré-existente que é bastante útil em certas situações.

É o decorador `@lru_cache` do módulo `functools`. Esse iterador pode ser usado em funções puras (isto é, que retornam sempre o mesmo resultado quando passamos os mesmos argumentos para elas e não fazem nenhuma alteração no estado global do programa ou operações de entrada e saída). **Ele não pode ser usado se a função tem efeitos colaterais!**  Ele é útil quando:

- O cálculo do valor da função pode ser demorado; e
- Espera-se que a função seja chamada frequentemente com os mesmos argumentos.

Vejamos um uso típico. Podemos implementar um código para calcular os número de Fibonacci de forma recursiva:

In [ ]:
def nth_fibonacci(n):
    if n == 0 or n == 1:
        return 1
    else:
        return nth_fibonacci(n - 1) + nth_fibonacci(n - 2)

Esta definição tem a vantagem de seguir diretamente a definição de números de Fibonacci, então temos facilmente certeza de que está correta (para n >= 0):

In [ ]:
nth_fibonacci(0), nth_fibonacci(1), nth_fibonacci(2), nth_fibonacci(3)

O problema é que a recursão da função gera um grande número de chamadas (duplicadas) da função, que crescem exponencialmente com o valor de `n`. Por exemplo, para `n=5` temos as seguintes chamadas (usando `f(n)` para indicar uma chamada com parâmetro `n`:

```
f(5):
   f(4):
     f(3):
       f(2):
         f(1): 1
         f(0): 1
       f(1): 1
     f(2):
       f(1): 1
       f(0): 1
   f(3):
     f(2):
       f(1): 1
       f(0): 1
     f(1): 1
```

Note o grande número de chamadas com valores repetidos. Mas como todas as chamadas com o mesmo valor de `n` vão ter valores de retorno iguais, podemos usar um _cache_. A idéia do uso de _cache_ é guardar um dicionário com as combinações de valores dos parâmetros e valores dos retornos correspondentes. Quando é feita uma chamada à função com um conjunto de parâmetros, primeiro verificamos se já temos esse parâmetro no dicionário, caso em que retornamos o valor imediatamente; se não temos o valor no dicionário, chamamos a função e então, antes de retornar o valor, guardamos a nova combinação parâmetros-valor de retorno para uso futuro.

Primeiro, vamos temporizar o código sem cache para um `n` não muito grande:

In [ ]:
%timeit nth_fibonacci(25)

E vejamos como ele cresce:

In [ ]:
%timeit nth_fibonacci(30)

Agora vamos usar o decorador `functools.lru_cache`:

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache(maxsize=None)
def nth_fibonacci(n):
    if n == 0 or n == 1:
        return 1
    else:
        return nth_fibonacci(n - 1) + nth_fibonacci(n - 2)

O uso de `maxsize=None` indica que vamos guardar todas as combinações de parâmetros usadas. Isso é possível neste caso, pois não esperamos que a função seja usada para muitos valores de `n` diferentes. Entretanto, se temos muitas possíveis combinações de parâmetros, podemos querer limitar o tamanho da cache, para evitar uso excessivo de memória. Veja a documentação.

E vejamos agora como fica o desempenho:

In [ ]:
%timeit nth_fibonacci(25)

In [ ]:
%timeit nth_fibonacci(30)

O tempo muito baixo reportado aqui é devido ao fato de que o `%timeit` realiza várias execuções do código para calcular médias (veja as mensagens). Mas como estamos usando um cache, apenas a primeira execução realmente toma algum tempo importante.

# Exercícios

1. Qual a saída produzida pelo código abaixo?
```python
def angry(f):
    def angry_f(x):
        s = f(x)
        return s.upper()
    return angry_f
       
@angry
def message(x):
    return f"I don't like {x}!"
       
print(message('pineapple pizza'))
```

2. Re-escreva o decorador `angry` do código acima para ele funcionar com funções que recebem quaisquer tipos e número de parâmetros.

3. Crie um decorador de classe, denominado `counted`, que retorna um versão da classe decorada que conta o número de objetos dessa classe gerados, guardando esse contador no atributo de classe `num_objects`. Um exemplo de uso seria:
```python
@counted
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'Point({self.x}, {self.y})'
       
o = Point(0, 0)
p1 = Point(1, 1)
p2 = Point(-1, 1)
print(f'We have the points {o}, {p1}, {p2}')
print(f'This is a total of {Point.num_objects} points')
```